In [ ]:
"""
@author: mdigi14
"""

import pandas as pd
import matplotlib.pyplot as plt

"""
Update Parameters Here
"""
FILE = "quaks"
ADDRESS = "0xd9d1c2623fbb4377d9bf29075e610a9b8b4805b4"  # set to "" if you want to print graphs for all addresses that minted more than MIN_TOKENS_MINTED
TOKEN_COL = 'TOKEN_ID'    
MIN_TOKENS_MINTED =  25

LOWER_BOUND = 0
UPPER_BOUND = 6000
TOP_N = 6000

"""
Plot params
"""
plt.rcParams.update({
    "figure.facecolor": "white",
    "savefig.facecolor": "white"
})

"""
Helper Functions
"""

def print_graph(grifter_address, MINTING_DB, RARITY_DB):
    GRIFTER_DB = MINTING_DB[MINTING_DB['to_account'] == grifter_address]

    tokens_minted = GRIFTER_DB['TOKEN_ID']
    rarity_ranks =  GRIFTER_DB['rank']

    if len(tokens_minted) >= MIN_TOKENS_MINTED or ADDRESS != "":

        if TOKEN_COL == 'TOKEN_NAME':
            RARITY_DB['TOKEN_ID'] = RARITY_DB['TOKEN_NAME'].str.split('#').str[1].astype(int)


        RARITY_DB = RARITY_DB.sort_values('TOKEN_ID')

        ax = RARITY_DB.plot.scatter(x='TOKEN_ID', y='Rank', grid=True, alpha=.25, title= "{} - {}".format(FILE, grifter_address), figsize=(14, 7))
        plt.scatter(x = tokens_minted, y=rarity_ranks, color="black", s=55)

        ax.set_xlabel("Token ID")
        ax.set_ylabel("Rarity Rank")
        plt.xlim(LOWER_BOUND, UPPER_BOUND+50)
        plt.ylim(0, TOP_N)
        plt.show()

In [ ]:
"""
Generate Plot
"""

PATH = "../metadata/rarity_data/{}_raritytools.csv".format(FILE)
MINT_PATH = "../minting_data/{}_minting.csv".format(FILE)      
RARITY_DB = pd.read_csv(PATH)
RARITY_DB = RARITY_DB[RARITY_DB['TOKEN_ID'].duplicated()==False]
MINTING_DB = pd.read_csv(MINT_PATH)

addresses = set(MINTING_DB['to_account'].unique())

if ADDRESS != "":
    print_graph(ADDRESS, MINTING_DB, RARITY_DB)
else:
    for address in addresses:
        print_graph(address,  MINTING_DB, RARITY_DB)